In [ ]:
# Install required packages
!pip install langchain langchain-community langchain-core langchain-huggingface chromadb pandas transformers langchain-google-genai gradio langid deep-translator

In [ ]:
import os
import pandas as pd
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint, HuggingFacePipeline
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
import torch

In [ ]:
# Set your HuggingFace token
os.environ["GOOGLE_API_KEY"] = "*******************************"
os.environ["HF_TOKEN"] = "******************************"
HF_TOKEN = os.environ["HF_TOKEN"]

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "dataset.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "birdy654/human-and-llm-mental-health-conversations",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

<ipython-input-4-df85d36c431c>:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 2.63M/2.63M [00:00<00:00, 154MB/s]

Extracting zip of dataset.csv...
First 5 records:                                              Context  \
0  I'm going through some things with my feelings...   
1  I'm going through some things with my feelings...   
2  I'm going through some things with my feelings...   
3  I'm going through some things with my feelings...   
4  I'm going through some things with my feelings...   

                                            Response  \
0  If everyone thinks you're worthless, then mayb...   
1  Hello, and thank you for your question and see...   
2  First thing I'd suggest is getting the sleep y...   
3  Therapy is essential for those that are feelin...   
4  I first want to let you know that you are not ...   

                                                 LLM  
0   I understand that you're feeling incredibly l...  
1   I'm sorry to hear that you've been feeling wo...  
2   I'm glad you've reached out to me today. I un...  
3   I'm sorry to hear that you're feeling this wa...  
4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/mental_health_chatbot_dataset.csv")
df_train = df_train.dropna(subset=["human", "assistant"])
df_train.head()

,human,assistant
0,What is a panic attack?,Panic attacks come on suddenly and involve int...
1,What are symptoms of panic attack vs. anxiety ...,Panic attacks and anxiety attacks can share so...
2,What are the types of Mental Illness?,There are many different conditions that are r...
3,What does mental-illness mean?,Mental illness is a group of biological brain ...
4,How can you treat mental illness?,The treatment of mental illness is a multi-fac...


In [ ]:
df_kz = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/mental_health_counseling_kazakh_translate.csv")
df_kz = df_kz.dropna(subset=["Context_kk", "Response_kk"])
df_kz = df_kz.drop(["Context", "Response"], axis=1)
df_kz.head()

,Context_kk,Response_kk
0,Кішкентайымнан бері басымды қабырғаларға және ...,Бұл деңгейдегі мазасыздықты емдеудің ең жақсы ...
1,Бір жылдан астам уақыт бұрын менің бір әйел до...,"Біз, әйелдер, салыстыру ойынымен күресуге бейі..."
2,Менің қашықтағы қызым студенттік қауымдастықта...,"Мүмкін, сіз қарым-қатынасыңыз үшін қолдан келг..."
3,"Маған опасыздық жасау мүлдем қабылданбайды, бі...","Сіздің осы опасыздықты ұмытып, кешіруге тырысы..."
4,Менде егіз сәбилер бар. Мен босанғанға дейін ...,"Біріншіден, сіздің аман қалған және жауынгер ..."


In [ ]:
def load_csv_as_documents(df, df_kz, df_train):
    """Loads a Pandas DataFrame and converts it to LangChain Documents."""
    docs = []
    for idx, row in df.iterrows():
        content = f"Context: {row['Context']}\nResponse: {row['Response']}\nLLM: {row['LLM']}"
        docs.append(Document(page_content=content, metadata={"row_index": idx}))
    return docs

    for idx, row in df_kz.iterrows():
        content = f"Context_kk: {row['Context_kk']}\Response_kk: {row['Response_kk']}"
        docs.append(Document(page_content=content, metadata={"row_index": idx}))
    return docs

    for idx, row in df_train.iterrows():
        content = f"human: {row['human']}\assistant: {row['assistant']}"
        docs.append(Document(page_content=content, metadata={"row_index": idx}))
    return docs

documents = load_csv_as_documents(df, df_kz, df_train)

#document chunking
def create_chunks(docs):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_documents(docs)

text_chunks = create_chunks(documents)

# Step 3: Embedding model
def get_embedding_model():
    return GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

embedding_model = get_embedding_model()

In [ ]:
# Add this after creating text_chunks
print("Sample chunk content:")
print(text_chunks[6004].page_content)  # Check several chunks

Sample chunk content:
this work for yourself and the other.  Also, if you are fully in and ready to do the work but your partner isn't.  If they say things like "this is just how I am" or "we have problems because of your issues" then you also may want to really slow down and consider if this will be workable.  You and your boyfriend need to be invested and committed to the relationship even when things are hard.


In [ ]:
# Add this after creating text_chunks
print("Sample chunk content:")
print(text_chunks[9094].page_content)  # Check several chunks

Sample chunk content:
the impact of his actions on you. Explore if there could be external factors causing his behavior, such as stress or work-related issues. If communication doesn't bring about the desired change, consider seeking the help of a professional counselor or therapist, who can provide additional insight and techniques to strengthen your relationship. In the meantime, make sure you are practicing self-care, connecting with friends and family, and engaging in enjoyable activities, as these things can


In [ ]:
# storing in ChromaDB
DB_CHROMA_PATH = "chroma_db"
vectordb = Chroma.from_documents(text_chunks, embedding_model, persist_directory=DB_CHROMA_PATH)
vectordb.persist()

<ipython-input-11-5b9ec771469f>:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [ ]:
# Test the retriever directly
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

test_retrieval = retriever.get_relevant_documents("sleep")
print("Retrieved documents:")
for doc in test_retrieval:
    print(doc.page_content)
    print("---")

Retrieved documents:
2. Create a sleep-conducive environment: Make sure your bedroom is dark, quiet, and cool, and invest in a comfortable mattress and bedding. Using a white noise machine or fans can help block out external noise.

3. Avoid screens and bright lights before bed: The blue light emitted by electronic devices can interfere with your circadian rhythms, making it harder to fall asleep. Try to avoid screens for at least an hour before bedtime and consider reading a book instead.
---
3. Engage in relaxing activities before bed: Avoid using electronic devices or watching TV right before bed. Instead, try activities such as reading, taking a warm bath, or practicing relaxation techniques, like deep breathing or meditation.
4. Limit caffeine and alcohol intake: These substances can disrupt your sleep, so try to avoid them, especially in the hours leading up to bedtime.
---
4. Practice relaxation techniques: Techniques like progressive muscle relaxation or deep breathing can help

<ipython-input-12-4010a8de3cc4>:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  test_retrieval = retriever.get_relevant_documents("sleep")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "deepseek-ai/deepseek-llm-7b-chat"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")
#load_llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
from deep_translator import GoogleTranslator
import langid

# language detection limited to English, Russian, Kazakh
def detect_language(text):
    langid.set_languages(['en', 'ru', 'kk'])
    lang, _ = langid.classify(text)
    if lang not in ['en', 'ru', 'kk']:
        return "unknown"
    return lang

# translation function
def translate_to_english(text, src_lang):
    if src_lang == 'en':
        return text
    return GoogleTranslator(source=src_lang, target='en').translate(text)

def translate_from_english(text, target_lang):
    if target_lang == 'en':
        return text
    return GoogleTranslator(source='en', target=target_lang).translate(text)


In [ ]:
def set_custom_prompt(): #тоже я хз
    from langchain.prompts import PromptTemplate
    template = """
        You are a compassionate therapist AI.

        Below is a conversation between you and a user. Use the provided therapist responses to guide your advice. Do not repeat or quote the context. Just give a supportive and direct answer.

        Context:
        {context}

        User: {question}
        Therapist: """

    return PromptTemplate(template=template, input_variables=["context", "question"])


#'''
#You are a supportive therapist AI designed to respond compassionately to users' messages.

#Instructions:
#- Give emotionally supportive, empathetic, and specific advice based on the question and therapist responses below.
#- Do NOT repeat the context or question.
#- Be warm, concise, and relevant.

#---
#Therapist responses: {context}

#Question: {question}

#Answer:
#'''

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0.7, #поменять хз
    max_new_tokens=512,
    return_full_text=False  # important to avoid echoing
)

load_llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


In [ ]:
query = input("ваш вопрос: ")
language = detect_language(query)
print("Detected language:", language)

if language == "unknown":
    print("Извините, но бот поддерживает только русский, английский и казахский языки.")
else:
    translated_query = translate_to_english(query, language)

    # Retrieve context
    retrieved_docs = retriever.get_relevant_documents(translated_query)
    context_text = "\n".join([doc.page_content for doc in retrieved_docs])

    # Format prompt
    prompt_template = set_custom_prompt()
    formatted_prompt = prompt_template.format(context=context_text, question=translated_query)

    # Generate response
    response = load_llm.invoke(formatted_prompt)
    response_text = response.strip()

    # Translate back
    final_output = translate_from_english(response_text, language)
    print("\n🧠 Model response:\n")
    print(final_output)

ваш вопрос: привет
Detected language: ru

🧠 Model response:

Привет! Как я могу вам помочь сегодня?


In [ ]:
import gradio as gr

def chat(message, history):
    # detect input language
    lang = detect_language(message)
    if lang == "unknown":
        return "Извините, но бот поддерживает только русский, английский и казахский языки."

    # translate to English if needed
    translated_query = translate_to_english(message, lang)

    # retriever
    retrieved_docs = retriever.get_relevant_documents(translated_query)
    context_text = "\n".join([doc.page_content for doc in retrieved_docs])

    # prompt
    prompt_template = set_custom_prompt()
    formatted_prompt = prompt_template.format(context=context_text, question=translated_query)

    # generate model response
    response = load_llm.invoke(formatted_prompt)
    response_text = response.strip()

    # translate response back
    final_response = translate_from_english(response_text, lang)
    return final_response

# gradio interface
iface = gr.ChatInterface(
    fn=chat,
    title="Psychological Support Chatbot",
    description="Ask in Kazakh, Russian, or English. The bot will respond in your\
     language with compassion.",
    type="messages"
)

iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://80287dfb3b04242118.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://80287dfb3b04242118.gradio.live
